In [102]:
import folium
import numpy as np
import pandas as pd
import scipy.stats as stats
import requests
import re
import matplotlib
%matplotlib inline
%load_ext autoreload
%autoreload 1
%aimport helpers
from bs4 import BeautifulSoup

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Import the data. Opening the csv file in excel then saving it as an excel file took care of the delimiter problems. Don't judge us.

In [9]:
grants = pd.read_excel('P3_GrantExport.xlsx', header=0, usecols=["University","Approved Amount"])
grants.describe()

,University,Approved Amount
count,50988,63969
unique,77,36455
top,Universität Zürich - ZH,data not included in P3
freq,6774,10910


Note that the count of the two columns is different, hinting at the presence of null values for University.

Remove them:

In [10]:
grants.dropna(inplace=True)
grants.describe()

,University,Approved Amount
count,50988,50988
unique,77,36072
top,Universität Zürich - ZH,10000
freq,6774,594


Looks better.

From looking a bit at the data we noticed some values that are not of any use for us:

For __University__:
  * _Nicht zuteilbar_ - Not available
  * _NPO (Biblioth., Museen, Verwalt.)_ - Non profit organisation (libraries, museums, administrations)
  * _Firmen/Privatwirtschaft_ - Private entreprises

For __Approved Amount__:
  * _data not included in P3_ - Self-explanatory

In [11]:
grants_filtered = grants.loc[(grants['University'] != "Nicht zuteilbar - NA") &
                            (grants['University'] != "NPO (Biblioth., Museen, Verwalt.) - NPO") &
                            (grants['University'] != 'Firmen/Privatwirtschaft - FP') &
                            (grants['Approved Amount'] != "data not included in P3")]
grants_filtered.describe()

,University,Approved Amount
count,46318,46318.0
unique,74,33887.0
top,Universität Zürich - ZH,10000.0
freq,6754,537.0


We want the sum of grants for each university:

In [12]:
grants_grouped = grants_filtered.groupby('University').sum()
grants_grouped.describe()

,Approved Amount
count,7.400000e+01
mean,1.674274e+08
std,4.478958e+08
min,8.000000e+03
25%,1.272432e+06
50%,4.858361e+06
75%,3.970232e+07
max,1.838237e+09


Get the localisation information for the universities. At first just save the result of the request in the dataframe, refine it later. The localisation is obtained using Google Maps Places API.

In [55]:
#no need to execute again - see below
grants_grouped_localised = grants_grouped.copy()
grants_grouped_localised['Raw localisation'] = grants_grouped_localised.apply(lambda x: helpers.split_and_request(x.name), axis=1)
grants_grouped_localised

,Approved Amount,Raw localisation
University,,
AO Research Institute - AORI,3.435621e+06,{'results': [{'icon': 'https://maps.gstatic.co...
Allergie- und Asthmaforschung - SIAF,1.916996e+07,{'results': [{'icon': 'https://maps.gstatic.co...
Berner Fachhochschule - BFH,3.102870e+07,{'results': [{'icon': 'https://maps.gstatic.co...
Biotechnologie Institut Thurgau - BITG,2.492535e+06,{'results': [{'icon': 'https://maps.gstatic.co...
Centre de rech. sur l'environnement alpin - CREALP,1.567678e+06,{'results': [{'icon': 'https://maps.gstatic.co...
EPF Lausanne - EPFL,1.175316e+09,{'results': [{'icon': 'https://maps.gstatic.co...
ETH Zürich - ETHZ,1.635597e+09,{'results': [{'icon': 'https://maps.gstatic.co...
Eidg. Anstalt für Wasserversorgung - EAWAG,7.461922e+07,{'results': [{'icon': 'https://maps.gstatic.co...
"Eidg. Forschungsanstalt für Wald,Schnee,Land - WSL",4.836039e+07,"{'results': [], 'html_attributions': [], 'stat..."


Save the results in a CSV file. Since the requests to the Places API are pretty expensive and Google only authorizes a small number of them every day, we want to avoid having to redo them.

In [96]:
grants_grouped_localised.to_csv('grants_grouped_localised.csv')

Let's see what information we got.

In [128]:
grants_grouped_localised.iloc[1]['Raw localisation']

{'html_attributions': [],
 'results': [{'formatted_address': 'Obere Str. 22, 7270 Davos Platz, Switzerland',
   'geometry': {'location': {'lat': 46.7954192, 'lng': 9.8200409},
    'viewport': {'northeast': {'lat': 46.79571619999999, 'lng': 9.82028165},
     'southwest': {'lat': 46.79532020000001, 'lng': 9.81931865}}},
   'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/generic_business-71.png',
   'id': 'ea435d78b7507556ed24683ca26f040f93a8d840',
   'name': 'Schweiz. Institut f. Allergie- u. Asthmaforschung',
   'place_id': 'ChIJL3RyxgGkhEcR24v33tCoi5A',
   'reference': 'CmRSAAAA_sQi97X0Joc7hnfvGUTfPRkVunGSZaRDRln_3nyhOiX7aECkD8v0J_LGKLvGKYKIg66mZpgTk6cvbLptJ7-9OhB3ZiWJn615h10vmFDxdURO3Vkr1qwre0rzbmTbXJujEhCs3BctOW__2A7bDqyFNxMcGhQ4ipq1XO8YvmQmeE7xM_ibGEq32Q',
   'types': ['point_of_interest', 'establishment']}],
 'status': 'OK'}

Most of this is not very interesting. Let's keep only the latitude and the longitude.

In [139]:
grants_with_cantons = grants_grouped.copy()
grants_with_cantons['Coordinates'] = grants_grouped_localised.apply(helpers.extract_lat_lng, axis = 1)
#now use that to get the cantons
grants_with_cantons['Canton'] = grants_with_cantons.apply(lambda x: helpers.canton_from_coordinates(x['Coordinates']), axis=1)

,Approved Amount
count,7.400000e+01
mean,1.674274e+08
std,4.478958e+08
min,8.000000e+03
25%,1.272432e+06
50%,4.858361e+06
75%,3.970232e+07
max,1.838237e+09


In [142]:
grants_with_cantons.head(5)

,Approved Amount,Coordinates,Canton
University,,,
AO Research Institute - AORI,3435621.0,"{'lng': 9.8136932, 'lat': 46.7771199}",GR
Allergie- und Asthmaforschung - SIAF,19169965.0,"{'lng': 9.8200409, 'lat': 46.7954192}",GR
Berner Fachhochschule - BFH,31028695.0,"{'lng': 7.621387700000001, 'lat': 47.0574225}",BE
Biotechnologie Institut Thurgau - BITG,2492535.0,"{'lng': 9.161303, 'lat': 47.64832730000001}",TG
Centre de rech. sur l'environnement alpin - CREALP,1567678.0,"{'lng': 7.367433499999999, 'lat': 46.2285589}",VS


Save it to avoid doing too many requests.

In [150]:
x = grants_with_cantons.to_csv('grants_with_cantons.csv')

In [3]:
grants_with_cantons = pd.read_csv('grants_with_cantons.csv', encoding='ISO-8859-1')

Show those for which we haven't found the localisation

In [14]:
null_locations = grants_with_cantons[grants_with_cantons['Coordinates'].isnull()][['University', 'Approved Amount']]
null_locations

,University,Approved Amount
8,"Eidg. Forschungsanstalt für Wald,Schnee,Land -...",48360389.63
10,Eidg. Material und Prüfungsanstalt - EMPA,58574515.92
13,Fachhochschule Nordwestschweiz (ohne PH) - FHNW,42771914.12
17,Forschungsanstalten Agroscope - AGS,33115719.00
20,Forschungskommission SAGW,100000.00
24,"Haute école pédagogique BE, JU, NE - HEPBEJUNE",627380.00
39,Physikal.-Meteorolog. Observatorium Davos - PMOD,12098436.00
40,Pädag. Hochschule Tessin (Teilschule SUPSI) - ASP,159317.00
56,Schweizer Kompetenzzentrum Sozialwissensch. - ...,34735816.00
58,Staatsunabh. Theologische Hochschule Basel - STHB,17300.00


Let's try with the Geocode API this time, might give us better results:

In [86]:
locations = null_locations['University'].apply(split_and_geocode)
locations

8     [{'types': ['bus_station', 'establishment', 'p...
10    [{'types': ['locality', 'political'], 'geometr...
13                                                 None
17    [{'types': ['administrative_area_level_1', 'po...
20                                                 None
24                                                 None
39    [{'types': ['locality', 'political'], 'address...
40    [{'types': ['administrative_area_level_1', 'po...
56    [{'types': ['premise'], 'geometry': {'location...
58    [{'types': ['locality', 'political'], 'address...
59    [{'types': ['locality', 'political'], 'geometr...
60    [{'types': ['locality', 'political'], 'geometr...
71                                                 None
72    [{'types': ['establishment', 'natural_feature'...
Name: University, dtype: object

This looks promising. Are they right though? Let's check if they are in Switzerland.

In [104]:
cantons = locations.apply(helpers.get_canton)
cantons

8       ZH
10    None
13    None
17    None
20    None
24    None
39      GR
40      TI
56    None
58      BS
59    None
60    None
71    None
72    None
Name: University, dtype: object

4 more. Well, there is no small profit. Let's add them to the others.

In [120]:
for i in cantons.index:
    if cantons[i] is not None:
        grants_with_cantons.set_value(i, 'Coordinates', locations[i][0]['geometry']['location'])
        grants_with_cantons.set_value(i, 'Canton', cantons[i])

Let's see what's left.

In [122]:
null_locations = grants_with_cantons[grants_with_cantons['Coordinates'].isnull()][['University', 'Approved Amount']]
null_locations

,University,Approved Amount
10,Eidg. Material und Prüfungsanstalt - EMPA,58574515.92
13,Fachhochschule Nordwestschweiz (ohne PH) - FHNW,42771914.12
17,Forschungsanstalten Agroscope - AGS,33115719.00
20,Forschungskommission SAGW,100000.00
24,"Haute école pédagogique BE, JU, NE - HEPBEJUNE",627380.00
56,Schweizer Kompetenzzentrum Sozialwissensch. - ...,34735816.00
59,Swiss Center for Electronics and Microtech. - ...,18068246.00
60,Swiss Institute of Bioinformatics - SIB,11583219.00
71,Weitere Institute - FINST,9256736.00
72,Weitere Spitäler - ASPIT,10749808.00


For these we will have to work manually.  
  * (10) This is in Dübendorf, Zürich. Coordinates: 47.402177, 8.618307. 
  * (13) Olten, so Solothurn. Probably didn't work because of the part between parenthesis. Coordinates: 47.348233, 7.907826
  * (17) This is in 4 different places. We don't know which is the main, so we drop it.
  * (20) This is the research commission of the _Schweizerische Akademie der Geistes- und Sozialwissenschaften_. It didn't work because the hyphen is missing so it didn't search for _SAGW_. It's in Bern. Coordinates: 46.947241, 7.436539
  * (24) This one is for 3 different cantons. It's based in Delémont, Jura, so we go with that. Coordinates: 47.362856, 7.351272
  * (56) This is an institute doing research on social sciences, hosted by UNIL.
  * (59) That one is in Neuchâtel. Probably didn't work because the name is usually in French. Coordinates: 46.997902, 6.947501
  * (60) Geneva. Probably also a problem of English. Coordinates: 46.193478, 6.151104
  * (71, 72) Those mean _More institutions_ and _More hospitals_. So we don't know to what they belong and we have to drop them.

In [135]:
helpers.set_coordinates_and_canton(grants_with_cantons, 10, 8.618307, 47.402177, 'ZH')
helpers.set_coordinates_and_canton(grants_with_cantons, 13, 7.907826, 47.348233, 'SO')
helpers.set_coordinates_and_canton(grants_with_cantons, 20, 7.436539, 46.947241, 'BE')
helpers.set_coordinates_and_canton(grants_with_cantons, 24, 7.351272, 47.362856, 'JU')
helpers.set_coordinates_and_canton(grants_with_cantons, 56, 6.5801606, 46.5210895, 'VD')
helpers.set_coordinates_and_canton(grants_with_cantons, 59, 6.947501, 46.997902, 'NE')
helpers.set_coordinates_and_canton(grants_with_cantons, 60, 6.151104, 46.193478, 'GE')

Now drop the ones still missing.

In [139]:
grants_with_cantons.dropna(inplace=True)

Check if there are some odd canton codes:

In [173]:
grants_with_cantons['Canton'][lambda x: x.str.len() != 2]

30    Genève
36     Lazio
68    Genève
Name: Canton, dtype: object

Lazio... Isn't that Rome?

In [175]:
grants_with_cantons.loc[36]

University                       Istituto Svizzero di Roma - ISR
Approved Amount                                           141000
Coordinates        {'lng': 12.4876114, 'lat': 41.90634319999999}
Canton                                                     Lazio
Name: 36, dtype: object

Well okay then. At least change the Genève to GE.

In [180]:
grants_with_cantons.set_value(30, 'Canton', 'GE')
grants_with_cantons.set_value(68, 'Canton', 'GE')

,University,Approved Amount,Coordinates,Canton
0,AO Research Institute - AORI,3.435621e+06,"{'lng': 9.8136932, 'lat': 46.7771199}",GR
1,Allergie- und Asthmaforschung - SIAF,1.916996e+07,"{'lng': 9.8200409, 'lat': 46.7954192}",GR
2,Berner Fachhochschule - BFH,3.102870e+07,"{'lng': 7.621387700000001, 'lat': 47.0574225}",BE
3,Biotechnologie Institut Thurgau - BITG,2.492535e+06,"{'lng': 9.161303, 'lat': 47.64832730000001}",TG
4,Centre de rech. sur l'environnement alpin - CR...,1.567678e+06,"{'lng': 7.367433499999999, 'lat': 46.2285589}",VS
5,EPF Lausanne - EPFL,1.175316e+09,"{'lng': 6.566757600000001, 'lat': 46.5190557}",VD
6,ETH Zürich - ETHZ,1.635597e+09,"{'lng': 8.547669899999999, 'lat': 47.376313}",ZH
7,Eidg. Anstalt für Wasserversorgung - EAWAG,7.461922e+07,"{'lng': 8.609547599999999, 'lat': 47.4037666}",ZH
8,"Eidg. Forschungsanstalt für Wald,Schnee,Land -...",4.836039e+07,"{'lng': 8.456272, 'lat': 47.360093}",ZH
9,Eidg. Hochschulinstitut für Berufsbildung - EHB,2.086572e+06,"{'lng': 7.450692999999999, 'lat': 47.001506}",BE


Save the final clean data

In [71]:
grants_with_cantons.to_csv('grants_with_cantons.csv')

Read it back

In [75]:
grants_with_cantons = pd.read_csv('grants_with_cantons.csv', encoding='ISO-8859-1', usecols=['University', 'Approved Amount', 'Coordinates', 'Canton'])

Change the _Coordinates_ back to a dict

In [76]:
import ast
grants_with_cantons['Coordinates'] = grants_with_cantons.apply(lambda row: ast.literal_eval(row['Coordinates']), axis=1)

In [77]:
grants_with_cantons.head(5)

,University,Approved Amount,Coordinates,Canton
0,AO Research Institute - AORI,3435621.0,"{'lat': 46.7771199, 'lng': 9.8136932}",GR
1,Allergie- und Asthmaforschung - SIAF,19169965.0,"{'lat': 46.7954192, 'lng': 9.8200409}",GR
2,Berner Fachhochschule - BFH,31028695.0,"{'lat': 47.0574225, 'lng': 7.621387700000001}",BE
3,Biotechnologie Institut Thurgau - BITG,2492535.0,"{'lat': 47.64832730000001, 'lng': 9.161303}",TG
4,Centre de rech. sur l'environnement alpin - CR...,1567678.0,"{'lat': 46.2285589, 'lng': 7.367433499999999}",VS


Testing the maps. First group the data and add some for missing ids.

In [6]:
test = grants_with_cantons.copy()
grouped_test = test[['Canton', 'Approved Amount']].groupby(by='Canton').sum()
#add the canton as a column, required for folium
grouped_test['Canton'] = grouped_test.index
#add values for missing cantons
grouped_test = grouped_test.append(pd.DataFrame({'Approved Amount': 0, 'Canton': ['UR', 'OW', 'NW', 'GL', 'BL', 'AR', 'AI']}, 
                                                index=['UR', 'OW', 'NW', 'GL', 'BL', 'AR', 'AI']))
grouped_test['Approved Amount'] = grouped_test['Approved Amount'].apply(lambda x: 0 if x == 0 else np.log10(x))
grouped_test.sort_values(by='Approved Amount')

,Approved Amount,Canton
Canton,,
AI,0.000000,AI
BL,0.000000,BL
GL,0.000000,GL
NW,0.000000,NW
OW,0.000000,OW
AR,0.000000,AR
UR,0.000000,UR
Lazio,5.149219,Lazio
SH,5.247214,SH


Let's try some maps.

If you want to try some colours, here are the possible color palettes: 'BuGn', 'BuPu', 'GnBu', 'OrRd', 'PuBu', 'PuBuGn', 'PuRd', 'RdPu', YlGn', 'YlGnBu', 'YlOrBr', and 'YlOrRd'

They correspond to the multi-hue here: http://colorbrewer2.org/#type=sequential&scheme=BuGn&n=3
Also, I chose the starting location at semi-rand

In [23]:
topo_path = r'ch-cantons.topojson.json'
grants_map = folium.Map(location=[46.8, 8.239], zoom_start=8)
grants_map.choropleth(geo_path = topo_path, data=grouped_test,
                     columns=['Canton', 'Approved Amount'], 
                     key_on='feature.id',
                     fill_color='PuRd', fill_opacity=0.7, line_opacity=0.2,
                     topojson='objects.cantons',
                     threshold_scale=[0, 5, 6, 7, 8, 9])
grants_map.save('map.html')
#grants_map

In [40]:
from IPython.display import HTML, display, Javascript

In [41]:
display(HTML("map.html"))

In [113]:
grants_by_canton.loc['GE']['Approved Amount']

1888684759.0999999

In [103]:
total_grants = grants_with_cantons['Approved Amount'].sum()
grants_by_canton = grants_with_cantons[['Canton', 'Approved Amount']].groupby('Canton').sum()


In [116]:
x = grants_with_cantons.apply(lambda row: 
                          helpers.create_marker(row, grants_by_canton.loc[row['Canton']]['Approved Amount'], total_grants).add_to(grants_map), 
                          axis=1)

In [117]:
grants_map